In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

In [0]:
%tensorflow_version 1.x

In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/gru_seq2seq_export',}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/gru_seq2seq_export/1572592036/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  print('Output Shape:', predictions['output'].shape)
  for j in range(3):
    print('A{}:'.format(j+1), ' '.join([params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j]]).replace('<end>', ''))
  print('Spent %.2f sec' % (time.time() - t0))
  print()

Input:你好
Output Shape: (1, 50, 3)
A1: 你 好                                                
A2: 你 好 啊                                               
A3: 你 好 呀                                               
Spent 1.17 sec

Input:早上好
Output Shape: (1, 9, 3)
A1: 早 上 好      
A2: 早 上 好 ！     
A3: 早        
Spent 0.08 sec

Input:晚上好
Output Shape: (1, 10, 3)
A1: 晚 上 好       
A2: 晚 上 好 ！      
A3: 晚 安        
Spent 0.08 sec

Input:再见
Output Shape: (1, 7, 3)
A1: 再 见 ！    
A2: 再 见     
A3: 再 见 ！ ！ ！  
Spent 0.06 sec

Input:好久不见
Output Shape: (1, 7, 3)
A1: 好 久 不 见   
A2: 好 久 没 见 过  
A3: 好 久     
Spent 0.06 sec

Input:想死你了
Output Shape: (1, 10, 3)
A1: 我 也 想 死 你 了    
A2: 我 也 想 死 你     
A3: 我 也 想 死 了     
Spent 0.08 sec

Input:谢谢你
Output Shape: (1, 50, 3)
A1: 不 用 谢 。                                              
A2: 不 客 气                                               
A3: 不 客 气 ~                                              
Spent 0.33 sec

Input:爱你
Output Shape: (1, 8, 3)
A1: 我 也 爱 你    
A2: 我 爱 你  